<a href="https://colab.research.google.com/github/btnwilson/EvolutionOfTensegrityBots/blob/main/MutatingRobots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Evolving Tensegrity Robots and Their Control Systems
This project explores the coevolution of neural controllers and the physical structures of tensegrity robots. A tensegrity robot consists of mass points connected by motorized springs, which produce movement when their activation is properly synchronized. Programming these robots is challenging due to the non-intuitive combinations of spring activations required for effective motion.

To address this, neural network controllers are evolved in simulation using evolutionary algorithms. This project extends these principles by simultaneously evolving the robots’ structural configurations to identify optimal designs for movement. The objective of the simulation is simple: maximize the robot's movement to the right.

The following code sets up a 2D physics simulation using Taichi and implements the control architecture, evolutionary loops, and other components of the system.

In [ ]:
#Install Taichi
!pip3 install taichi==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00


In [ ]:
# import necessary modules
import taichi as ti
import numpy as np
import os
from IPython.display import HTML
from base64 import b64encode
import math
import random

#define simulation settings
max_steps = 200
dt = 0.01
ground_height = 0.125
gravity = -9.8
stiffness_value = 1050.0
damping = 0.1
learning_rate = 1.0

# create a dictionary for the genotype
genotype = {}

#lists for initialization of masses and springs
startingObjectPositions = []
springs = []

# Create an evenly spaced 2d grid of springs, a total of 16
for x in range(1, 5):
  for y in range(0, 4):
    startingObjectPositions.append([(x/10),ground_height + (y/10)])

# define an adjacency dictionary to describe the possible connections from each mass point
adjacency_list = {"0": [1, 4, 5], "1": [0, 2, 4, 5, 6], "2": [1, 3, 5, 6, 7], "3": [2, 6, 7],
                  "4": [0, 1, 5, 8, 9], "5": [0, 1, 2, 4, 6, 8, 9, 10], "6": [1, 2, 3, 5, 7, 9, 10, 11],
                  "7": [2, 3, 6, 10, 11], "8": [4, 5, 9, 12, 13], "9": [4, 5, 6, 8, 10, 12, 13, 14],
                  "10": [5, 6, 7, 9, 11, 13, 14, 15], "11": [6, 7, 10, 14, 15], "12": [8, 9, 13],
                  "13": [8, 9, 10, 12, 14], "14": [9, 10, 11, 13, 15], "15": [10, 11, 14]}

# declare a set of all possible connections and fill it using the adjacency table
all_connections = set()
for node, neighbors in adjacency_list.items():
    for neighbor in neighbors:
        connection = tuple(sorted([int(node), neighbor]))
        all_connections.add(connection)

# initialize a random robot body configuration and ensure that it is not disjointed and is adequately connected
meets_conditions = False
while not meets_conditions:
  is_disjoint = True
  has_too_few_connections = True

  # randomly sample between 5 and 12 mass points
  selected_nodes = random.sample(list(np.arange(0,17)), random.randint(5, 13))

  #select a random combination of springs that use the mass points selected above
  selected_connections = set()
  for node, neighbors in adjacency_list.items():
      if int(node) in selected_nodes:
          for neighbor in neighbors:
              if neighbor in selected_nodes:
                prob = random.uniform(0,1)
                if prob < .75:
                  selected_connections.add(tuple(sorted([int(node), neighbor])))

  selected_connections = list(selected_connections)

  # ensure all masses are connected to other masses
  test_node = selected_nodes[0]
  connected_set = []
  for i in range(len(selected_connections)):
    if i == 0:
      for j in selected_connections[0]:
        connected_set.append(j)
    else:
      for j in selected_connections:
        if j[0] in connected_set and j[1] not in connected_set:
          connected_set.append(j[1])
        elif j[1] in connected_set and j[0] not in connected_set:
          connected_set.append(j[0])

  # check the disjoint set condition
  if sorted(connected_set) == sorted(selected_nodes):
    is_disjoint = False

  # count the number of springs attached to each mass point
  num_of_connections_per_node = {}
  for node in selected_nodes:
    num_of_connections_per_node[node] = 0
  for connection in selected_connections:
    for node in connection:
      num_of_connections_per_node[node] += 1

  # Ensure each mass has at least 3 connections and less than 7. This guarentees a certain level of stability in robot mechanics.
  if min(num_of_connections_per_node.values()) > 2 and max(num_of_connections_per_node.values()) < 7:
    has_too_few_connections = False

  if not has_too_few_connections and not is_disjoint:
    meets_conditions = True

# print the selected nodes and connections and store them as lists in a genotype dictionary
print(selected_nodes)
print(selected_connections)
genotype["Parent"] = {}
genotype["Parent"]["nodes"] = selected_nodes
genotype["Parent"]["springs"] = selected_connections
genotype["Parent"]["fitness"] = 0.0
genotype["Child"] = {}
genotype["Child"]["nodes"] = []
genotype["Child"]["springs"] = []
genotype["Child"]["fitness"] = 0.0

# Initialize all springs connecting the original grid of masses but if they were not selected set their stiffness to zero
for i in all_connections:
  objAIndex = i[0]
  objAPos = startingObjectPositions[objAIndex]
  objBIndex = i[1]
  objBPos = startingObjectPositions[objBIndex]
  xDistAB = objAPos[0] - objBPos[0]
  yDistAB = objAPos[1] - objBPos[1]
  distAB = np.sqrt(xDistAB**2 + yDistAB**2)
  restingLength = distAB
  if i in selected_connections:
    springs.append([objAIndex, objBIndex, restingLength, 1, stiffness_value])
  else:
    springs.append([objAIndex, objBIndex, restingLength, 1, 0])

# use the minimum index number of nodes where the lower the number the closer to the bottom left of the field as the base value for loss function
loss_element = min(selected_nodes)

# variables for the number of springs and masses
n_objects = len(startingObjectPositions)
n_springs = len(springs)

# Initialize taichi data arrays needed for simulation
real = ti.f32
ti.init(default_fp = real)
spring_anchor_a = ti.field(ti.i32)
spring_anchor_b = ti.field(ti.i32)
spring_resting_lengths = ti.field(ti.f32)
spring_actuation = ti.field(ti.i32)
spring_stiffness = ti.field(ti.f32)

vec = lambda: ti.Vector.field(2, dtype=real)

positions = vec()

velocities = vec()

springRestoringForces = vec()

springForcesOnObjects = vec()

weightsSH = ti.field(ti.f32)
weightsSH_parent = ti.field(ti.f32)

# Initialize controller variables
n_sin_waves = 10
n_hidden_neurons = 32
center = vec()
goal = vec()
hidden = ti.field(ti.f32)
biasH = ti.field(ti.f32)
biasH_parent = ti.field(ti.f32)

# function to return the number of sensors that are on the robot
def n_sensors():
  return n_sin_waves + 4 * n_objects + 2

# finish initialization of the taichi data objects for variables using in physics simulation
weightsHM = ti.field(ti.f32)
weightsHM_parent = ti.field(ti.f32)
actuation = ti.field(ti.f32)

ti.root.dense(ti.i , max_steps).dense(ti.j , n_objects).place(positions)
ti.root.dense(ti.i , max_steps).dense(ti.j , n_objects).place(positions.grad)
ti.root.dense(ti.i , max_steps).dense(ti.j , n_objects).place(velocities)
ti.root.dense(ti.i, n_springs).place(spring_anchor_a, spring_anchor_b, spring_resting_lengths, spring_actuation, spring_stiffness)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_springs).place(springRestoringForces)
ti.root.dense(ti.i, max_steps).dense(ti.j, n_objects).place(springForcesOnObjects)
ti.root.dense(ti.ij, (n_hidden_neurons, n_sensors())).place(weightsSH)
ti.root.dense(ti.i, max_steps).place(center)
ti.root.place(goal)
ti.root.dense(ti.ij, (max_steps, n_hidden_neurons)).place(hidden)
ti.root.dense(ti.i, n_hidden_neurons).place(biasH)
ti.root.dense(ti.ij, (n_springs, n_hidden_neurons)).place(weightsHM)
ti.root.dense(ti.ij, (max_steps, n_springs)).place(actuation)
ti.root.dense(ti.ij, (n_hidden_neurons, n_sensors())).place(weightsSH_parent)
ti.root.dense(ti.ij, (n_springs, n_hidden_neurons)).place(weightsHM_parent)
ti.root.dense(ti.i, n_hidden_neurons).place(biasH_parent)

# value for loss and activation of gradient calculations
loss = ti.field(dtype = ti.f32, shape=())
ti.root.lazy_grad()

#----------------------------------------------------------------------------------------
def Mutate_Robot():
    '''
    This function mutates an existing robot to generate a new child phenotype.
    It alters the global genotype dictionary by applying one of three mutation
    types:
    1. Removing a connection (spring) between mass points.
    2. Adding a mass point and connecting it with springs.
    3. Removing a mass point along with its associated connections.

    The function ensures that the resulting child robot meets the following criteria:
    - The robot is not disjoint (all nodes are connected).
    - Each node has a minimum of 3 and a maximum of 6 connections.

    Parameters:
    None

    Returns:
    None
    '''
    # Flag to determine if the mutated robot meets all criteria
    child_meets_reqs = False

    # Continue mutation attempts until requirements are met
    while not child_meets_reqs:
        # Copy the current best robot genotype as a starting point
        child_nodes = genotype["Parent"]["nodes"].copy()
        child_connections = genotype["Parent"]["springs"].copy()

        # Initialize criteria flags
        is_disjoint = True
        has_too_few_connections = True

        # Randomly choose a mutation type (1 = remove connection, 2 = add node, 3 = remove node)
        mutation = random.randint(1, 3)

        # Mutation type 1: Remove a random connection (spring)
        if mutation == 1:
            child_connections.remove(random.choice(child_connections))

        # Mutation type 2: Add a new node and connect it to existing nodes
        elif mutation == 2:
            node_can_connect = False
            # Identify nodes that are not currently part of the robot
            possible_new_nodes = [i for i in np.arange(0, 16) if i not in child_nodes]

            # Ensure the new node can form at least one valid connection
            while not node_can_connect:
                new_node = random.choice(possible_new_nodes)
                child_nodes.append(new_node)
                original_num_connections = len(child_connections)

                # Add springs to the new node based on adjacency list
                for i in adjacency_list[f"{new_node}"]:
                    if i in child_nodes:
                        child_connections.append(tuple(sorted([new_node, i])))

                # Verify if at least one connection was added
                if original_num_connections < len(child_connections):
                    node_can_connect = True

        # Mutation type 3: Remove an existing node and its associated connections
        elif mutation == 3:
            new_child_connections = []
            removed_node = random.choice(child_nodes)
            child_nodes.remove(removed_node)

            # Retain only connections that do not involve the removed node
            for connection in child_connections:
                if removed_node not in connection:
                    new_child_connections.append(connection)
            child_connections = new_child_connections

        # Check if the resulting robot is disjoint
        connected_set = []
        for i in range(len(child_connections)):
            if i == 0:
                for j in child_connections[0]:
                    connected_set.append(j)
            else:
                for j in child_connections:
                    if j[0] in connected_set and j[1] not in connected_set:
                        connected_set.append(j[1])
                    elif j[1] in connected_set and j[0] not in connected_set:
                        connected_set.append(j[0])

        # A robot is not disjoint if all nodes are part of the connected set
        if sorted(connected_set) == sorted(child_nodes):
            is_disjoint = False

        # Calculate the number of connections per node and check constraints
        num_of_connections_per_node = {node: 0 for node in child_nodes}
        for connection in child_connections:
            for node in connection:
                num_of_connections_per_node[node] += 1

        # Ensure each node has at least 3 and no more than 6 connections
        if min(num_of_connections_per_node.values()) > 2 and max(num_of_connections_per_node.values()) < 7:
            has_too_few_connections = False

        # Set the flag if all criteria are met
        if not has_too_few_connections and not is_disjoint:
            child_meets_reqs = True

    # Update the genotype with the newly mutated child robot
    genotype["Child"]["nodes"] = child_nodes
    genotype["Child"]["springs"] = child_connections



#----------------------------------------------------------------------------------------
@ti.kernel
def Compute_Loss():
    '''
    Computes and updates the global loss term for the simulation.
    The loss is based on the horizontal distance (rightward movement) of
    a specific point that started at the bottom left corner of the system.
    The point's final horizontal position at the end of the simulation is used
    to calculate the loss value.

    The loss term is decreased by the final x-coordinate of the specified point
    (referred to as `loss_element`), indicating how far it has moved horizontally
    by the end of the simulation.

    Parameters:
    None

    Returns:
    None
    '''
    # Decrease the global loss by the x-coordinate of the point's final position
    # (i.e., how far the point has moved to the right)
    loss[None] -= positions[max_steps-1, loss_element][0]

    # Alternatively, if using the center of the robot, the loss could be computed as:
    # loss[None] -= center[max_steps-1]



#----------------------------------------------------------------------------------------
@ti.kernel
def Calculate_Center(timeStep: ti.i32):
    '''
    Calculates the geometric center of the tensegrity bot at a given time step
    and stores it in the `center` array.

    Parameters:
    -----------
    timeStep : ti.i32
        The current time step for which the center is calculated.

    Returns:
    --------
    None
    '''
    # Initialize a single loop iteration (dummy loop to enforce structure for Taichi kernels)
    for _ in range(1):
        # Initialize a vector to accumulate positions of all objects
        c = ti.Vector([0.0, 0.0])

        # Sum the positions of all objects at the given time step
        for i in ti.static(range(n_objects)):
            c += positions[timeStep, i]

        # Compute the average position (center of mass-like) and store in the center array
        center[timeStep] = c / n_objects

        # Alternatively, you could write the division explicitly for clarity
        # center[timeStep] = (1.0 / n_objects) * c


#----------------------------------------------------------------------------------------
def Draw(frameOffset):
    '''
    Draws the simulation in a simple 2D field using black and white graphics.
    - Masses are represented as circles.
    - Springs connecting the masses are represented as lines.
    - Different spring properties (e.g., stiffness) are visually distinguished by color and line thickness.

    Parameters:
    -----------
    frameOffset : int
        An offset value to adjust the frame numbering when saving the simulation images.

    This function iterates through each timestep of the simulation and:
    - Draws the ground line.
    - Draws the masses (nodes) based on their positions.
    - Draws the springs between the masses with visual properties based on their stiffness values.
    - Saves each frame as a PNG image with a unique filename.
    '''
    # Loop through each timestep in the simulation
    for timeStep in range(0, max_steps):
        # Initialize the GUI with a white background
        gui = ti.GUI("robot", (512, 512), background_color=0xFFFFFF, show_gui=False)

        # Draw the ground as a horizontal line across the field
        gui.line(begin=(0, ground_height), end=(1, ground_height), color=0x0, radius=3)

        # Loop through each object (node/mass)
        for objectIndex in range(n_objects):
            # Check if the object is part of the selected nodes
            if objectIndex in selected_nodes:
                # Get the x and y position of the node at the current timestep
                x = positions[timeStep, objectIndex][0]
                y = positions[timeStep, objectIndex][1]

                # Draw the node as a black circle
                gui.circle((x, y), color=0x0, radius=7)

        # Loop through each spring connecting nodes
        for springIndex in range(n_springs):
            # Get the indices of the nodes connected by the spring
            objectAIndex = spring_anchor_a[springIndex]
            objectBIndex = spring_anchor_b[springIndex]

            # Get the positions of the nodes at the current timestep
            posA = positions[timeStep, objectAIndex]
            posB = positions[timeStep, objectBIndex]

            # Check the spring's stiffness value
            if springs[springIndex][4] == stiffness_value:
                # Draw the spring as a black line if it matches the stiffness value
                gui.line(begin=posA, end=posB, color=0x0, radius=3)
            else:
                # Otherwise, draw the spring as a gray line
                gui.line(begin=posA, end=posB, color=0x808080, radius=1)

            # Note: The following block is commented out but might be used for further spring visualization
            """
            motorInside = springs[springIndex][3]
            if motorInside:
                gui.line(begin=posA, end=posB, color=0x0, radius=3)
            else:
                gui.line(begin=posA, end=posB, color=0x0, radius=1)
            """

        # Save the current frame as a PNG image with a unique filename
        gui.show(f"test{str(frameOffset + timeStep)}.png")


#----------------------------------------------------------------------------------------
def Initialize():
    '''
    Initializes the simulation by setting up initial conditions for object positions, velocities,
    spring properties, and arrays used for forces and actuation throughout the simulation.

    Key Actions:
    -----------
    1. Sets the initial positions and velocities of objects (masses).
    2. Configures spring properties such as anchor points, resting lengths, actuation values, and stiffness.
    3. Initializes position and velocity arrays for all timesteps to zero.
    4. Resets all force-related arrays (spring restoring forces, spring forces on objects) to zero.
    5. Sets up additional spring-related arrays (`hidden`, `actuation`) to zero.
    6. Defines a goal point for the simulation (used in optimization or as a target).

    Parameters:
    -----------
    None

    Returns:
    --------
    None
    '''
    # Set initial positions and velocities for each object
    for objectIndex in range(n_objects):
        # Position is initialized from starting positions
        positions[0, objectIndex] = startingObjectPositions[objectIndex]
        # Initial velocity is set to a small downward motion
        velocities[0, objectIndex] = [0, -0.1]

    # Initialize spring properties
    for springIndex in range(n_springs):
        s = springs[springIndex]
        # Set the indices of the objects connected by the spring
        spring_anchor_a[springIndex] = s[0]
        spring_anchor_b[springIndex] = s[1]
        # Set the spring's resting length, actuation value, and stiffness
        spring_resting_lengths[springIndex] = s[2]
        spring_actuation[springIndex] = s[3]
        spring_stiffness[springIndex] = s[4]

    # Reset positions and velocities for all future timesteps to zero
    for i in range(1, max_steps):
        for j in range(n_objects):
            positions[i, j][0] = 0.0
            positions[i, j][1] = 0.0
            velocities[i, j][0] = 0.0
            velocities[i, j][1] = 0.0

    # Reset spring restoring forces for all timesteps to zero
    for i in range(max_steps):
        for j in range(n_springs):
            springRestoringForces[i, j][0] = 0.0
            springRestoringForces[i, j][1] = 0.0

    # Reset spring forces on objects for all timesteps to zero
    for i in range(max_steps):
        for j in range(n_springs):
            springForcesOnObjects[i, j][0] = 0.0
            springForcesOnObjects[i, j][1] = 0.0

    # Reset additional spring-related arrays (`hidden` and `actuation`) to zero
    for i in range(max_steps):
        for j in range(n_springs):
            hidden[i, j] = 0.0
    for i in range(max_steps):
        for j in range(n_springs):
            actuation[i, j] = 0.0

    # Define the goal position for the simulation
    goal[None] = [0.9, 0.2]
#----------------------------------------------------------------------------------------
def Initialize_Neural_Network_Parameters():
    '''
    Initializes the parameters for the neural network controlling the simulation.

    Key Actions:
    -----------
    1. Sets random weights between sensors and hidden neurons (sensor-to-hidden connections).
    2. Sets random biases for the hidden neurons.
    3. Initializes random weights between hidden neurons and springs (hidden-to-motor connections).
    4. Defines the goal position for the simulation.

    Parameters:
    -----------
    None

    Returns:
    --------
    None
    '''
    # Initialize weights from sensors to hidden neurons
    for i in range(n_hidden_neurons):
        for j in range(n_sensors()):
            # Random weights in the range [-0.1, 0.1]
            weightsSH[i, j] = np.random.randn() * 0.2 - 0.1

    # Initialize biases for hidden neurons
    for i in range(n_hidden_neurons):
        # Random biases in the range [-1, 1]
        biasH[i] = np.random.randn() * 2 - 1

    # Define the goal position for the simulation
    goal[None] = [0.9, 0.2]

    # Initialize weights from hidden neurons to motors (springs)
    for i in range(n_springs):
        for j in range(n_hidden_neurons):
            # Random weights in the range [-0.1, 0.1]
            weightsHM[i, j] = np.random.randn() * 0.2 - 0.1

#----------------------------------------------------------------------------------------
def Make_Movie():
    '''
    Generates a movie (video) from a sequence of image files using the ffmpeg tool.

    This function first removes any existing video file named 'movie.mp4' in the working directory,
    and then uses ffmpeg to compile the images (named as 'test1.png', 'test2.png', ..., 'testN.png')
    into a single video file called 'movie.mp4'. The images are assumed to be in sequential order.

    Parameters:
    None

    Returns:
    None
    '''
    # Remove any existing movie file to avoid overwriting errors
    os.system('rm movie.mp4')

    # Use ffmpeg to combine images into a video. The images should be named test1.png, test2.png, etc.
    os.system("ffmpeg -i test%d.png movie.mp4")

#----------------------------------------------------------------------------------------
def Run_Simulation():
    '''
    Runs a simulation by initializing parameters, performing the simulation, and computing the loss.

    This function begins by initializing the robot and its components. Then, it runs the simulation
    where the robot's movements and interactions are computed. Afterward, the loss is computed based
    on the robot's performance, specifically the distance it has moved by the end of the simulation.

    The function also uses Taichi's automatic differentiation tool (`ti.Tape`) to track the loss and
    optimize it during the simulation.

    Parameters:
    None

    Returns:
    loss_value : float
        The computed loss value that indicates the robot's performance in the simulation.
    '''
    # Initialize robot and environment parameters
    Initialize()

    # Start tracking the loss with the 'ti.Tape' to enable backpropagation and optimization
    with ti.Tape(loss):

        # Perform the simulation (this will update robot states like position and velocity)
        Simulate()

        # Reset the loss to 0 before computing the final value
        loss[None] = 0

        # Compute the loss based on the final simulation state (e.g., robot's position)
        Compute_Loss()

    # Output the computed loss for the simulation
    print(loss[None])

    # Return the final loss value for further use (e.g., optimization or analysis)
    loss_value = loss[None]
    return loss_value


#----------------------------------------------------------------------------------------
def Simulate():
    '''
    Runs the simulation for a specified number of time steps, invoking the first simulation step for each time step.

    This function iterates through the time steps, from 1 to `max_steps`, and calls the function `Step_One`
    at each time step. The `Step_One` function likely handles the updates for the robot's state such as
    position, velocity, or other relevant physical properties.

    Parameters:
    None

    Returns:
    None
    '''
    # Iterate over each time step from 1 to max_steps to simulate the movement
    for timeStep in range(1, max_steps):
        # Execute the first step of the simulation at the current time step
        Step_One(timeStep)


#----------------------------------------------------------------------------------------
@ti.kernel
def Simulate_Neural_Network_HM(timeStep: ti.i32):
    '''
    Simulates the activation of the neural network and updates the actuation values for the springs.

    This function computes the activation for each spring in the tensegrity robot using a neural network-like
    architecture, where each spring's activation is determined by the weighted sum of hidden neuron values.
    The computed activation is then passed through a tanh activation function, which is assigned to the corresponding
    spring's actuation at the given time step.

    Parameters:
    timeStep: ti.i32
      The current time step in the simulation. The neural network activations and actuation values are computed for this time step.

    Returns:
    None
    '''
    # Iterate through each spring in the system
    for i in range(n_springs):

        # Initialize the activation value to zero for each spring
        activation = 0.0

        # Sum the weighted contributions of the hidden neurons to the activation value
        for j in ti.static(range(n_hidden_neurons)):
            activation += weightsHM[i, j] * hidden[timeStep, j]

        # Apply the tanh activation function to the summed value
        activation = ti.tanh(activation)

        # Store the computed activation value in the actuation array for the current time step and spring
        actuation[timeStep, i] = activation


#----------------------------------------------------------------------------------------
@ti.kernel
def Simulate_Neural_Network_SH(timeStep: ti.i32):
    '''
    Simulates the activation of the hidden layer neurons in the neural network using a combination of sinusoidal
    inputs and positional data from the robot's mass points. The computed activation values are then passed through
    a tanh activation function and stored in the hidden layer for the given time step.

    This function calculates the activation for each hidden neuron by considering sinusoidal inputs and the
    positional data of the robot, as well as an offset from the robot's center. The resulting activation is then
    processed and stored for the current time step.

    Parameters:
    timeStep: ti.i32
      The current time step in the simulation. The neural network activations for the hidden neurons are computed for this time step.

    Returns:
    None
    '''
    # Iterate through each hidden neuron in the network
    for i in range(n_hidden_neurons):
        # Initialize the activation value to zero for each hidden neuron
        activation = 0.0

        # Add sinusoidal inputs to the activation value
        for j in ti.static(range(n_sin_waves)):
            activation += weightsSH[i, j] * ti.sin(50 * timeStep * dt + 2 * math.pi / n_sin_waves * j)

        # Add positional data inputs to the activation value
        for j in ti.static(range(n_objects)):
            offset = positions[timeStep, j] - center[timeStep]
            # Include positional offsets in both x and y directions
            activation += 0.1 * weightsSH[i, j * 4 + n_sin_waves] * offset[0]
            activation += 0.1 * weightsSH[i, j + 4 + 1 + n_sin_waves] * offset[1]

            # Also include the current positions in both x and y directions
            activation += 0.1 * weightsSH[i, j * 4 + n_sin_waves] * positions[timeStep, j][0]
            activation += 0.1 * weightsSH[i, j + 4 + 1 + n_sin_waves] * positions[timeStep, j][1]

        # Add goal-related inputs to the activation value
        activation += 0.1 * weightsSH[i, n_objects * 4 + n_sin_waves] * (goal[None][0] - center[timeStep][0])
        activation += 0.1 * weightsSH[i, n_objects * 4 + n_sin_waves] * (goal[None][1] - center[timeStep][1])

        # Add the bias term for the hidden neuron
        activation += biasH[i]

        # Apply the tanh activation function
        activation = ti.tanh(activation)

        # Store the computed activation in the hidden layer for the current time step
        hidden[timeStep, i] = activation


#----------------------------------------------------------------------------------------
@ti.kernel
def Simulate_Objects(timeStep: ti.i32):
    '''
    Simulates the movement of objects in the system, updating their positions and velocities at each time step.
    The objects are affected by gravity, damping, and spring forces. Additionally, if an object reaches the ground
    and its velocity is directed downwards, its velocity is reset to zero.

    The function updates the positions and velocities of the objects for the given time step, considering external
    forces and interactions such as gravity and spring forces. The new position is computed based on the velocity,
    and the velocity is updated considering the damping and external forces.

    Parameters:
    timeStep: ti.i32
      The current time step in the simulation. The positions and velocities of objects are updated for this time step.

    Returns:
    None
    '''
    # Iterate through each object in the system
    for objectIndex in range(n_objects):

        # Get the position of the object at the previous time step
        oldPosition = positions[timeStep-1, objectIndex]

        # Calculate the velocity at the previous time step, considering damping, gravity, and spring forces
        oldVelocity = (1 - damping) * velocities[timeStep-1, objectIndex] + dt * gravity * ti.Vector([0, 1]) \
                      + springForcesOnObjects[timeStep, objectIndex]

        # If the object is below or at the ground and moving downwards, reset its velocity
        if oldPosition[1] <= ground_height and oldVelocity[1] < 0:
            oldVelocity = ti.Vector([0, 0])  # Set velocity to zero (object stops)

        # Compute the new position of the object using the updated velocity
        newPosition = oldPosition + dt * oldVelocity

        # Update the position of the object for the current time step
        positions[timeStep, objectIndex] = newPosition

        # The new velocity is the same as the computed old velocity (since there's no additional change)
        newVelocity = oldVelocity

        # Update the velocity of the object for the current time step
        velocities[timeStep, objectIndex] = newVelocity


#----------------------------------------------------------------------------------------
@ti.kernel
def Simulate_Springs(timeStep: ti.i32):
    '''
    Simulates the forces exerted by springs on the objects they connect. At each time step, the function calculates
    the restoring force of each spring, which depends on the displacement from its resting length, stiffness,
    and the current actuation. These forces are then applied to the connected objects.

    The function accounts for the actuation of springs, adjusting the resting length based on the actuation,
    and updates the forces on the connected objects. The force applied to each object is proportional to the
    spring's restoring force.

    Parameters:
    timeStep: ti.i32
      The current time step in the simulation. The spring forces are calculated and applied for this time step.

    Returns:
    None
    '''
    # Iterate through each spring in the system
    for springIndex in range(n_springs):
        # Get the indices of the two objects connected by the spring
        objectAIndex = spring_anchor_a[springIndex]
        objectBIndex = spring_anchor_b[springIndex]

        # Get the positions of the objects at the previous time step
        posA = positions[timeStep-1, objectAIndex]
        posB = positions[timeStep-1, objectBIndex]

        # Calculate the displacement vector between the two objects
        dist = posA - posB

        # Get the length of the spring (the norm of the displacement vector)
        length = dist.norm()

        # Get the spring's resting length and stiffness
        springRestingLength = spring_resting_lengths[springIndex]
        stiffness = spring_stiffness[springIndex]

        # Adjust the resting length based on the spring's actuation
        springRestingLength += spring_actuation[springIndex] * 0.02 * actuation[timeStep, springIndex]

        # Calculate the unhappiness (stretch/compression) of the spring
        springUnhappiness = length - springRestingLength

        # Calculate the spring's restoring force
        springRestoringForces[timeStep, springIndex] = (dt * stiffness * springUnhappiness / length) * dist

        # Apply the restoring force to the objects connected by the spring
        springForcesOnObjects[timeStep, objectAIndex] += -5.0 * springRestoringForces[timeStep, springIndex]
        springForcesOnObjects[timeStep, objectBIndex] += 5.0 * springRestoringForces[timeStep, springIndex]


#----------------------------------------------------------------------------------------
def Step_One(timeStep: ti.i32):
    '''
    Executes the first step of the simulation for the current time step. This function is responsible for
    updating various components of the simulation, including the center of the tensegrity bot, the neural
    network simulation (both SH and HM), the spring forces, and the movement of the objects.

    Parameters:
    timeStep: ti.i32
      The current time step in the simulation.

    Returns:
    None
    '''
    # Calculate the center of the tensegrity bot and store it for the current time step
    Calculate_Center(timeStep)

    # Simulate the neural network with the SH (Sine-based) configuration for actuation
    Simulate_Neural_Network_SH(timeStep)

    # Simulate the neural network with the HM (Hidden-layer-based) configuration for actuation
    Simulate_Neural_Network_HM(timeStep)

    # Calculate the restoring forces of the springs based on object positions and spring parameters
    Simulate_Springs(timeStep)

    # Update the positions and velocities of the objects based on forces, damping, and gravity
    Simulate_Objects(timeStep)


#----------------------------------------------------------------------------------------
def Tune_HM_Weights():
    '''
    Updates the weights of the HM (Hidden-layer-to-Springs) neural network by performing
    gradient descent. The function adjusts the weights in the direction that minimizes
    the loss based on the gradient of the weights for each spring and hidden neuron pair.

    This function assumes that gradients have been computed during the backward pass
    (via a previous forward pass with the `ti.Tape` context) and uses these gradients
    to update the weights of the HM network.

    Returns:
    None
    '''
    # Iterate over each spring in the system
    for i in range(n_springs):
        # Iterate over each hidden neuron in the network
        for j in range(n_hidden_neurons):
            # Update the weights using gradient descent rule:
            # weight[i,j] -= learning_rate * gradient[i,j]
            weightsHM[i, j] -= learning_rate * weightsHM.grad[i, j]


#----------------------------------------------------------------------------------------
def Tune_Hidden_Layer_Biases():
    '''
    Updates the biases of the hidden layer in the neural network by performing gradient descent.
    The function adjusts the biases in the direction that minimizes the loss based on the gradient
    of the biases for each hidden neuron.

    This function assumes that gradients for the biases have been computed during the backward pass
    (via a previous forward pass with the `ti.Tape` context) and uses these gradients to update the biases.

    Returns:
    None
    '''
    # Iterate over each hidden neuron in the network
    for i in range(n_hidden_neurons):
        # Update the bias
        biasH[i] -= learning_rate * biasH.grad[i]


#----------------------------------------------------------------------------------------
def Tune_SH_Weights():
    '''
    Updates the weights of the SH layer in the neural network by performing gradient descent.
    This function adjusts the weights based on the gradient of the weights for each hidden neuron
    and its connections to sine wave inputs and object-specific weights.

    The function assumes that gradients for the weights have been computed during the backward pass
    (via a previous forward pass with the `ti.Tape` context) and uses these gradients to update the weights.

    Returns:
    None
    '''
    # Iterate over each hidden neuron in the network
    for i in range(n_hidden_neurons):
        # Update the weights for sine wave connections
        for j in ti.static(range(n_sin_waves)):
            # Perform gradient descent update for each sine wave weight
            weightsSH[i, j] -= learning_rate * weightsSH.grad[i, j]

        # Update the weights for object-related connections
        for j in ti.static(range(n_objects)):
            # Perform gradient descent update for the weights associated with object connections
            # There are two weight indices for each object-related connection (for each coordinate)
            weightsSH[i, j * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i, j * 4 + n_sin_waves]
            weightsSH[i, j + 4 + 1 + n_sin_waves] -= learning_rate * weightsSH.grad[i, j + 4 + 1 + n_sin_waves]

            # Repeat update for specific object-related connections (possibly a redundant line)
            weightsSH[i, j * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i, j * 4 + n_sin_waves]
            weightsSH[i, j + 4 + 1 + n_sin_waves] -= learning_rate * weightsSH.grad[i, j + 4 + 1 + n_sin_waves]

        # Update the weight for the final object-related connection (goal)
        weightsSH[i, n_objects * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i, n_objects * 4 + n_sin_waves]
        weightsSH[i, n_objects * 4 + n_sin_waves] -= learning_rate * weightsSH.grad[i, n_objects * 4 + n_sin_waves]


#----------------------------------------------------------------------------------------
def Tune_Robots_Brain():
    '''
    Updates the parameters (weights and biases) of the neural network associated with the robot's brain.
    This function tunes the weights for both the Hidden Layer (HM) and the Sine-wave related connections (SH),
    and it also adjusts the hidden layer biases. The updates are done by performing gradient descent using
    the gradients computed during the backward pass.

    This function is called to optimize the neural network parameters during training. It is intended to be
    executed after each simulation step to continuously improve the robot's performance based on the loss function.

    Returns:
    None
    '''
    # Update the weights for the Hidden Layer connections (HM)
    Tune_HM_Weights()

    # Update the biases of the hidden layer neurons
    Tune_Hidden_Layer_Biases()

    # Update the weights for the Sine-wave related connections (SH)
    Tune_SH_Weights()


#----------------------------------------------------------------------------------------
def Copy_Weights_To_Genome():
    '''
    Copies the current robot's neural network weights and biases to the genome (genotype) of the child robot.
    This function saves the weight matrices for the Hidden Layer connections (HM), the Sine-wave related
    connections (SH), and the biases to the child robot's genotype, effectively creating a genetic representation
    of the robot's neural network parameters.

    The child robot's genotype can later be used to initialize its neural network during training or reproduction.

    Returns:
    None
    '''
    # Initialize numpy arrays to hold the weights and biases for the child robot
    weights_HM_robot = np.zeros((n_springs, n_hidden_neurons))
    bias_robot = np.zeros((n_hidden_neurons))
    weights_SH_robot = np.zeros((n_hidden_neurons, n_sin_waves))

    # Copy the Hidden Layer weights (HM) from the parent to the child robot
    for i in range(n_springs):
        for j in range(n_hidden_neurons):
            weights_HM_robot[i, j] = weightsHM_parent[i, j]

    # Copy the hidden layer biases from the parent to the child robot
    for i in range(n_hidden_neurons):
        bias_robot[i] = biasH_parent[i]

    # Copy the Sine-wave weights (SH) from the parent to the child robot
    for i in range(n_hidden_neurons):
        for j in ti.static(range(n_sin_waves)):
            weights_SH_robot[i, j] = weightsSH_parent[i, j]

    # Save the child robot's weights and biases to its genotype
    genotype["Child"]["bias"] = bias_robot
    genotype["Child"]["sh"] = weights_SH_robot
    genotype["Child"]["hm"] = weights_HM_robot


#----------------------------------------------------------------------------------------
def Copy_Weights_To_Parent():
    '''
    Copies the current robot's neural network weights and biases from the child robot to the parent robot.
    This function updates the parent's weights and biases with the values from the child robot's neural network.

    This process is typically used in an evolutionary context, where the parent robot takes on the parameters
    of the child after reproduction or learning to continue evolving or training.

    Returns:
    None
    '''
    # Copy the Hidden Layer weights (HM) from the child robot to the parent robot
    for i in range(n_springs):
        for j in range(n_hidden_neurons):
            weightsHM_parent[i, j] = weightsHM[i, j]

    # Copy the Sine-wave weights (SH) from the child robot to the parent robot
    for i in range(n_hidden_neurons):
        for j in ti.static(range(n_sin_waves)):
            weightsSH_parent[i, j] = weightsSH[i, j]

    # Copy the hidden layer biases from the child robot to the parent robot
    for i in range(n_hidden_neurons):
        biasH_parent[i] = biasH[i]


#----------------------------------------------------------------------------------------
def Copy_Weights_To_Working():
    '''
    Copies the parent robot's neural network weights and biases to the working robot (current state) for simulation or training.
    This function is used to transfer the latest neural network parameters from the parent to the working robot
    before performing a simulation step or applying updates.

    Returns:
    None
    '''
    # Copy the Hidden Layer weights (HM) from the parent robot to the working robot
    for i in range(n_springs):
        for j in range(n_hidden_neurons):
            weightsHM[i, j] = weightsHM_parent[i, j]

    # Copy the Sine-wave weights (SH) from the parent robot to the working robot
    for i in range(n_hidden_neurons):
        for j in ti.static(range(n_sin_waves)):
            weightsSH[i, j] = weightsSH_parent[i, j]

    # Copy the hidden layer biases from the parent robot to the working robot
    for i in range(n_hidden_neurons):
        biasH[i] = biasH_parent[i]


#----------------------------------------------------------------------------------------
# Running the actual simulation

# Lists to store fitness values during the simulation
final_fitnesses = []
midtrial_fitnesses = []

# Outer loop for simulating multiple mutation steps
for mutationStep in range(10):  # Runs the mutation step 10 times
    Initialize_Neural_Network_Parameters()  # Initialize neural network parameters at the start of each mutation step
    loss_parent = 0  # Variable to track the parent loss (fitness)
    skip_reduct = False  # Flag to control the learning rate reduction
    network_training_fitness = []  # List to store fitness values during the training of the current mutation step

    # Inner loop for optimizing (training) the robot brain for a fixed number of steps (50)
    for optimizationStep in range(50):
        print(learning_rate)  # Print the current learning rate for each optimization step

        # Run the simulation and get the loss value (fitness) from it
        loss_value = Run_Simulation()

        # If it's the first optimization and mutation step, handle initial setup
        if optimizationStep == 0 and mutationStep == 0:
            if mutationStep == 0:
                loss_parent = loss_value  # Set initial loss for the parent
                os.system('rm test*.png')  # Remove any old images (if any)
                Draw(0)  # Draw the initial state
                Copy_Weights_To_Parent()  # Copy initial weights to parent
                Tune_Robots_Brain()  # Tune the robot's brain (neural network)
            else:
                # If it's not the first mutation, just copy weights and tune the robot's brain
                Copy_Weights_To_Parent()
                Tune_Robots_Brain()
        else:
            # If the current loss is better than the parent, update and set the learning rate to 1.0
            if loss_value < loss_parent:
                print("Child was more fit")  # Output if child robot is fitter
                Copy_Weights_To_Parent()  # Copy weights to parent
                loss_parent = loss_value  # Update parent loss value
                learning_rate = 1.0  # Set learning rate to 1.0 for better exploration
            else:
                # If the child robot is less fit, reduce learning rate or reset
                print("Child was less fit")  # Output if child robot is less fit
                Copy_Weights_To_Working()  # Copy weights back to the working robot

                # Reduce the learning rate if it's above 0.1
                if learning_rate > 0.1 and skip_reduct == False:
                    learning_rate = learning_rate - 0.05
                else:
                    learning_rate = 1.0  # Reset learning rate back to 1.0
                    if skip_reduct == True:
                        skip_reduct = False  # Reset skip flag after one reduction
                    else:
                        skip_reduct = True  # Skip reduction on the next iteration
                        loss_parent = loss_value  # Update loss for the parent

        # Tune the robot's brain (neural network weights) for the current iteration
        Tune_Robots_Brain()
        print("")  # Print a blank line for clarity

        # Append the current loss (fitness) to the training fitness list
        network_training_fitness.append(loss_parent)

    # After the inner loop ends (i.e., training for this mutation step), add the fitness values
    midtrial_fitnesses.append(network_training_fitness)

    # Update the working robot's weights after training
    Copy_Weights_To_Working()

    # Run the final simulation for the current mutation step
    Run_Simulation()

    # Optionally, you can save the genome of the robot (this is commented out)
    # Copy_Weights_To_Genome()

    # Draw the results for the current mutation step
    Draw(max_steps * (mutationStep+1))

    # Append the final loss (fitness) for the current mutation step to final_fitnesses
    final_fitnesses.append(loss_parent)

    # Update the fitness values for parent and child robots
    if mutationStep == 0:
        genotype["Parent"]["fitness"] = loss_parent  # Set initial parent fitness
    else:
        genotype["Child"]["fitness"] = loss_parent  # Set current child fitness

    # If the parent's fitness is better than the child's, update parent's nodes and springs
    if genotype["Parent"]["fitness"] > genotype["Child"]["fitness"]:
        genotype["Parent"]["nodes"] = genotype["Child"]["nodes"].copy()
        genotype["Parent"]["springs"] = genotype["Child"]["springs"].copy()
        genotype["Parent"]["fitness"] = genotype["Child"]["fitness"]

    # Mutate the robot's parameters to create a new child robot
    Mutate_Robot()

    # Copy selected nodes and connections from the child's genotype to be used for simulation
    selected_nodes = genotype["Child"]["nodes"].copy()
    selected_connections = genotype["Child"]["springs"].copy()

    # Print selected nodes and connections for debugging
    print(selected_nodes)
    print(selected_connections)

    # Initialize the positions of the starting objects
    startingObjectPositions = []
    springs = []  # List to hold the spring connections between objects

    # Set the starting positions for the objects (grid-like structure)
    for x in range(1, 5):
        for y in range(0, 4):
            startingObjectPositions.append([(x/10), ground_height + (y/10)])

    # Set up the spring connections between the objects
    for i in all_connections:
        objAIndex = i[0]
        objAPos = startingObjectPositions[objAIndex]
        objBIndex = i[1]
        objBPos = startingObjectPositions[objBIndex]
        xDistAB = objAPos[0] - objBPos[0]
        yDistAB = objAPos[1] - objBPos[1]
        distAB = np.sqrt(xDistAB**2 + yDistAB**2)  # Calculate distance between objects
        restingLength = distAB  # Set the spring's resting length
        if i in selected_connections:
            springs.append([objAIndex, objBIndex, restingLength, 1, stiffness_value])
        else:
            springs.append([objAIndex, objBIndex, restingLength, 1, 0])  # Non-selected springs have 0 stiffness

    # Select the node with the minimum value (for loss calculation or other purpose)
    loss_element = min(selected_nodes)

    # Set the number of objects and springs for the simulation
    n_objects = len(startingObjectPositions)
    n_springs = len(springs)

    # Set initial learning rate for the next mutation step
    learning_rate = 1.0

# After the simulation loop, make a movie from the simulation results
Make_Movie()

# Read the generated movie file and encode it as base64 for embedding
mp4 = open("movie.mp4", 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Display the video within the notebook or HTML page
HTML('<video width=512 controls> <source src="%s" type="video/mp4"></video>' % data_url)



[6, 11, 10, 14, 5, 2, 4, 1, 15, 0]
[(5, 10), (0, 5), (1, 6), (11, 14), (2, 5), (10, 15), (6, 11), (4, 5), (5, 6), (14, 15), (0, 1), (1, 2), (0, 4), (10, 11), (1, 5), (10, 14), (6, 10), (1, 4), (2, 6), (11, 15)]
[Taichi] Starting on arch=x64
1.0
-0.12033872306346893

1.0
-0.12795467674732208
Child was more fit

1.0
-0.11478842794895172
Child was less fit

0.95
-0.12049918621778488
Child was less fit

0.8999999999999999
-0.09029387682676315
Child was less fit

0.8499999999999999
-0.1237749308347702
Child was less fit

0.7999999999999998
-0.10259709507226944
Child was less fit

0.7499999999999998
-0.11758624017238617
Child was less fit

0.6999999999999997
-0.10473937541246414
Child was less fit

0.6499999999999997
-0.12881341576576233
Child was more fit

1.0
-0.10175375640392303
Child was less fit

0.95
-0.11277428269386292
Child was less fit

0.8999999999999999
-0.10064071416854858
Child was less fit

0.8499999999999999
-0.1172766163945198
Child was less fit

0.7999999999999998
-0.104883

In [ ]:
print(midtrial_fitnesses)
print(final_fitnesses)

[[-0.12033872306346893, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12795467674732208, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.12881341576576233, -0.11941614747047424, -0.11941614747047424, -0.12913116812705994, -0.12913116812705994, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028757274150848, -0.13028